<h1> Creación de Modelos

<h3> Creación de modelos en base a máscaras y a puntos de interés

Equipo de Reto 5

Inteligencia artificial avanzada para la ciencia de datos II (Grupo 502)

30 de noviembre de 2023

Modificacion cambiando parametros de la GPU y utilizando generators para evitar problemas de RAM

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

### Preparar el entorno de trabajo

In [2]:
# Importar librerias y módulos necesarios
import os
import numpy as np
from keras.layers import Conv2D, Activation, MaxPooling2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.models import load_model
import tensorflow as tf
import random
import shutil
# from google.colab.patches import cv2_imshow
import cv2

2023-11-30 11:27:54.022499: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-30 11:27:54.066832: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-11-30 11:27:56.714688: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:27:56.763550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:27:56.763759: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [5]:
# Definir el directorio de trabajo actual
os.chdir('/home/alanr/Documents/Corazon')

In [6]:
# Definir el directorio con los conjuntos de datos
data_directory = os.path.join(os.getcwd(), 'CompleteDatasets')

In [7]:
# Definir el directorio para guardar los mejores modelos y su historial de entrenamiento
# Definir el directorio para guardar los mejores modelos y su historial de entrenamiento
models_directory = os.path.join(os.getcwd(), 'Models')
logs_directory = os.path.join(os.getcwd(), 'Logs')



In [8]:
# Definir dimensiones importantes de los datos
img_height = 112
img_width = 112
img_channels = 1
mask_channels = 1
landmarks_channels = 7

### Definir la arquitectura para un modelo U-Net

In [9]:
# Definir un bloque convolucional
def conv_block(input, num_filters):
  x = Conv2D(num_filters, 3, padding='same')(input)
  x = Activation('relu')(x)
  x = Conv2D(num_filters, 3, padding='same')(x)
  x = Activation('relu')(x)
  return x

In [10]:
# Definir un bloque codificador
def encoder_block(input, num_filters):
  x = conv_block(input, num_filters)
  p = MaxPooling2D((2,2))(x)
  return x, p

In [11]:
# Definir un bloque decodificador
def decoder_block(input, skip_features, num_filters):
  x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(input)
  x = Concatenate()([x, skip_features])
  x = conv_block(x, num_filters)
  return x

In [12]:
# Definir la arquitectura para un modelo U-Net
def build_unet(input_shape, num_clases):
  inputs = Input(input_shape)

  s1, p1 = encoder_block(inputs, 64)
  s2, p2 = encoder_block(p1, 128)
  s3, p3 = encoder_block(p2, 256)
  s4, p4 = encoder_block(p3, 512)

  b1 = conv_block(p4, 1024)

  d1 = decoder_block(b1, s4, 512)
  d2 = decoder_block(d1, s3, 256)
  d3 = decoder_block(d2, s2, 128)
  d4 = decoder_block(d3, s1, 64)

  outputs = Conv2D(num_clases, 1, padding='same', activation='sigmoid')(d4)

  model = Model(inputs, outputs, name='U-Net')
  return model

### Crear y entrenar un modelo U-Net basado en máscaras

In [13]:
# # Cargar el conjunto de entrenamiento
# train_input = np.load(os.path.join(data_directory, 'train_images.npy'))
# train_output = np.load(os.path.join(data_directory, 'train_masks.npy'))
# print('Dimensiones del conjunto de entrenamiento:', train_input.shape, train_output.shape)

# # Cargar el conjunto de validación
# val_input = np.load(os.path.join(data_directory, 'val_images.npy'))
# val_output = np.load(os.path.join(data_directory, 'val_masks.npy'))
# print('Dimensiones del conjunto de validación   :', val_input.shape, val_output.shape)

In [14]:
# # Crear y compilar el modelo basado en máscaras
# mask_model = build_unet((img_height, img_height, img_channels), mask_channels)
# mask_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# # Definir "Callbacks" para guardar los modelos mejor entrenados y el historial de entrenamiento
# checkpoint = ModelCheckpoint(os.path.join(models_directory, 'masks_{epoch:02d}_{val_accuracy:.2f}.h5'), monitor='val_accuracy', save_best_only=True, mode='max')
# log_csv = CSVLogger(os.path.join(logs_directory, 'MaskModels.csv'), separator=',', append=False)

In [16]:
# with tf.device('/cpu:0'):
#     train_input = tf.convert_to_tensor(train_input, np.float32)
#     train_output = tf.convert_to_tensor(train_output, np.float32)

#     val_input = tf.convert_to_tensor(val_input, np.float32)
#     val_output = tf.convert_to_tensor(val_output, np.float32)
   
# define generator function
# def generator_images_and_masks():
#     for idx in range(len(train_input)):
#         # extract one image and the corresponding mask
#          img = train_input[idx]
#          mask = train_output[idx]

#          # convert to TF tensors
#          img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
#          mask_tensor = tf.convert_to_tensor(mask, dtype=tf.float32)

#          yield img_tensor, mask_tensor

# def generator_images_and_masks_val():
#     for idx in range(len(val_input)):
#         # extract one image and the corresponding mask
#          img = val_input[idx]
#          mask = val_output[idx]

#          # convert to TF tensors
#          img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
#          mask_tensor = tf.convert_to_tensor(mask, dtype=tf.float32)

#          yield img_tensor, mask_tensor

# # create dataset using generator function and specifying shapes and dtypes
# dataset_train = tf.data.Dataset.from_generator(generator_images_and_masks, 
#                                          output_signature=(tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32),
#                                                            tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32)))

# dataset_val = tf.data.Dataset.from_generator(generator_images_and_masks_val, 
#                                          output_signature=(tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32),
#                                                            tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32)))



In [17]:
# # Explicitly set the batch size
# batch_size = 16
# dataset_train = dataset_train.batch(batch_size)
# dataset_val = dataset_val.batch(batch_size)

In [18]:
# # Entrenar el modelo con los datos de entrenamiento y validación
# mask_model.fit(dataset_train, validation_data=dataset_val,callbacks=[checkpoint, log_csv], epochs=10)

In [19]:
# mask_model.save("20Random_dataAug_mask_model.h5")

### Crear y entrenar un modelo U-Net basado en puntos de interés

In [20]:
# Cargar el conjunto de entrenamiento
train_input = np.load(os.path.join(data_directory, 'train_images.npy'))
train_input = train_input[:-1]
train_output = np.load(os.path.join(data_directory, 'train_landmarks.npy'))
print('Dimensiones del conjunto de entrenamiento:', train_input.shape, train_output.shape)

# Cargar el conjunto de validación
val_input = np.load(os.path.join(data_directory, 'val_images.npy'))
val_output = np.load(os.path.join(data_directory, 'val_landmarks.npy'))
print('Dimensiones del conjunto de validación:   ', val_input.shape, val_output.shape)

Dimensiones del conjunto de entrenamiento: (15342, 112, 112, 1) (15342, 112, 112, 7)
Dimensiones del conjunto de validación:    (2644, 112, 112, 1) (2644, 112, 112, 7)


In [21]:
# Crear y compilar el modelo basado en puntos de interés
landmark_model = build_unet((img_height, img_height, img_channels), landmarks_channels)
landmark_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2023-11-30 11:27:58.905095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:27:58.905465: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-30 11:27:58.905674: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [22]:
# Definir "Callbacks" para guardar los modelos mejor entrenados y el historial de entrenamiento
checkpoint = ModelCheckpoint(os.path.join(models_directory, 'landmarks_{epoch:02d}_{val_accuracy:.2f}.h5'), monitor='val_accuracy', save_best_only=True, mode='max')
log_csv = CSVLogger(os.path.join(logs_directory, 'LandmarkModels.csv'), separator=',', append=False)

In [23]:
# with tf.device('/cpu:0'):
#     train_input = tf.convert_to_tensor(train_input, np.float32)
#     train_output = tf.convert_to_tensor(train_output, np.float32)

#     val_input = tf.convert_to_tensor(val_input, np.float32)
#     val_output = tf.convert_to_tensor(val_output, np.float32)
   
# define generator function
def generator_images_and_masks():
    for idx in range(len(train_input)):
        # extract one image and the corresponding mask
         img = train_input[idx]
         mask = train_output[idx]

         # convert to TF tensors
         img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
         mask_tensor = tf.convert_to_tensor(mask, dtype=tf.float32)

         yield img_tensor, mask_tensor

def generator_images_and_masks_val():
    for idx in range(len(val_input)):
        # extract one image and the corresponding mask
         img = val_input[idx]
         mask = val_output[idx]

         # convert to TF tensors
         img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
         mask_tensor = tf.convert_to_tensor(mask, dtype=tf.float32)

         yield img_tensor, mask_tensor

# create dataset using generator function and specifying shapes and dtypes
dataset_train = tf.data.Dataset.from_generator(generator_images_and_masks, 
                                         output_signature=(tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32),
                                                           tf.TensorSpec(shape=(112, 112, 7), dtype=tf.float32)))

dataset_val = tf.data.Dataset.from_generator(generator_images_and_masks_val, 
                                         output_signature=(tf.TensorSpec(shape=(112, 112, 1), dtype=tf.float32),
                                                           tf.TensorSpec(shape=(112, 112, 7), dtype=tf.float32)))



In [24]:
# Explicitly set the batch size
batch_size = 16
dataset_train = dataset_train.batch(batch_size)
dataset_val = dataset_val.batch(batch_size)

In [25]:
# Entrenar el modelo con los datos de entrenamiento y validación
# Entrenar el modelo con los datos de entrenamiento y validación
landmark_model.fit(dataset_train, validation_data=dataset_val,callbacks=[checkpoint, log_csv], epochs=10)

Epoch 1/10


2023-11-30 11:28:01.220593: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-11-30 11:28:01.771925: W tensorflow/tsl/framework/bfc_allocator.cc:366] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2023-11-30 11:28:02.890567: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0b6f0ef3a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-30 11:28:02.890617: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Comput

    958/Unknown - 212s 208ms/step - loss: 0.0212 - accuracy: 0.1390

2023-11-30 11:31:34.174498: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-30 11:31:38.600514: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


    959/Unknown - 220s 216ms/step - loss: 0.0212 - accuracy: 0.1391

2023-11-30 11:31:53.551160: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10161103296151599331
2023-11-30 11:31:53.551300: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16564640189053613321
2023-11-30 11:31:53.551337: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8079921698153951001
/usr/lib/python3/dist-packages/keras/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


959/959 [==============================] - 234s 230ms/step - loss: 0.0212 - accuracy: 0.1391 - val_loss: 0.0144 - val_accuracy: 0.2152
Epoch 2/10
959/959 [==============================] - 215s 224ms/step - loss: 0.0145 - accuracy: 0.2398 - val_loss: 0.0140 - val_accuracy: 0.1924
Epoch 3/10
959/959 [==============================] - 217s 227ms/step - loss: 0.0139 - accuracy: 0.2566 - val_loss: 0.0140 - val_accuracy: 0.2623
Epoch 4/10
959/959 [==============================] - 215s 224ms/step - loss: 0.0136 - accuracy: 0.2619 - val_loss: 0.0142 - val_accuracy: 0.3109
Epoch 5/10
959/959 [==============================] - 210s 219ms/step - loss: 0.0134 - accuracy: 0.2839 - val_loss: 0.0141 - val_accuracy: 0.2920
Epoch 6/10
959/959 [==============================] - 209s 218ms/step - loss: 0.0133 - accuracy: 0.2935 - val_loss: 0.0140 - val_accuracy: 0.2664
Epoch 7/10
959/959 [==============================] - 209s 218ms/step - loss: 0.0132 - accuracy: 0.2918 - val_loss: 0.0140 - val_accura

In [26]:
landmark_model.save("20Random_dataAug_landmark_model.h5")

### Probar el funcionamiento de los mejores modelos con el conjunto de prueba

In [27]:
# Borrar los conjuntos de entrenamiento y validación tras entrenar los modelos
train_input = train_output = val_input = val_output = 0

# Cargar el conjunto de prueba
test_images = np.load(os.path.join(data_directory, 'test_images.npy'))
print('Dimensiones del conjunto de prueba:', test_images.shape)

Dimensiones del conjunto de prueba: (2598, 112, 112, 1)


In [28]:
# Cargar los mejores modelos
mask_model = load_model(os.path.join(models_directory, 'masks_08_0.99.h5'), compile=False)
landmark_model = load_model(os.path.join(models_directory, 'landmarks_05_0.45.h5'), compile=False)

OSError: No file or directory found at /home/alanr/Documents/Corazon/Models/masks_08_0.99.h5

In [ ]:
# Predecir con ambos modelos todos los datos en el conjunto de prueba
mask_predictions = mask_model.predict(test_images)
landmark_predictions = landmark_model.predict(test_images)

In [ ]:
# Mostrar predicciones para datos aleatorios del conjunto de prueba
predictions = []

# Para k número de predicciones
for index in random.choices(range(0, len(test_images)), k=5):

  # Seleccionar la imagen original y convertir a tres canales
  frame = test_images[index]
  frame = np.concatenate([frame, frame, frame], axis=2)

  # Añadir la máscara sobre la imagen original
  mask_prediction = np.squeeze(mask_predictions[index] > 0.5)
  frame[:,:,0] = cv2.addWeighted(mask_prediction.astype(np.float32), 0.75, frame[:,:,0].astype(np.float32), 1, 0)

  # Añadir los puntos de interés sobre la imagen original
  landmarks_prediction = landmark_predictions[index]
  background = np.zeros_like(frame[:,:,0])
  for channel in range(landmarks_prediction.shape[2]):
    landmark = (landmarks_prediction[:,:,channel] >= np.max(landmarks_prediction[:,:,channel]))
    coord = np.unravel_index(np.argmax(landmark), landmark.shape)
    coord = (coord[1], coord[0])
    cv2.circle(background, coord, 2, 255, -1)
  frame[:,:,2] = cv2.addWeighted(background.astype(np.float32), 0.75, frame[:,:,2].astype(np.float32), 1, 0)

  # Añadir la imagen a una lista
  predictions.append(frame * 255)

# Mostrar en una sola línea las imagenes originales con sus predicciones
divider = np.full((img_height, 1, 3), 255)
cv2_imshow(np.concatenate((predictions[0], divider, predictions[1], divider, predictions[2], divider, predictions[3], divider, predictions[4]), axis=1))